In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import pandas as pd
from utils import prepare_df, group_cases_by_trace, compute_accuracies_by_case
from incremental_softmax_recovery import incremental_softmax_recovery

In [4]:
# Configure logging with selective DEBUG for our modules only
logging.basicConfig(
    level=logging.INFO,           # Set root to INFO (reduces third-party noise)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True                    # Force override of any existing handlers (useful in Jupyter)
)

# Enable DEBUG for our specific modules only
our_modules = [
    'classes', 
    'incremental_softmax_recovery', 
    'utils', 
    'conformance_checking',
    'data_processing',
    'petri_model',
    'calibration'
]

for module_name in our_modules:
    logging.getLogger(module_name).setLevel(logging.DEBUG)

# Silence noisy third-party libraries
logging.getLogger('graphviz').setLevel(logging.WARNING)  # Only show warnings/errors from graphviz
logging.getLogger('matplotlib').setLevel(logging.WARNING)  # Silence matplotlib if present
logging.getLogger('PIL').setLevel(logging.WARNING)  # Silence PIL if present

print("✅ Logging configured: DEBUG for our modules, INFO+ for third-party libraries")


✅ Logging configured: DEBUG for our modules, INFO+ for third-party libraries


In [5]:
# load your DataFrame and softmax list
result = prepare_df('50salads')
if len(result) == 2:
    df, softmax_lst = result
else:
    df, softmax_lst, _ = result

# group by trace and inspect
trace_groups = group_cases_by_trace(df)
trace_groups

,case_list,trace_length
0,"[0, 1, 2, 3]",5687
1,"[32, 33, 34, 35]",6186
2,"[36, 37, 38, 39]",5840
3,"[28, 29, 30, 31]",5261
4,"[4, 5, 6, 7]",6208
5,"[16, 17, 18, 19]",6293
6,"[24, 25, 26, 27]",6046
7,"[8, 9, 10, 11]",6584
8,"[12, 13, 14, 15]",5558
9,"[20, 21, 22, 23]",5792


In [ ]:
# Configuration for incremental_softmax_recovery (conformance-only)
config = {
    # === Data Splitting ===
    'n_train_traces': 10,
    'n_test_traces': 10,
    'train_cases': None,
    'test_cases': None,
    'ensure_train_variant_diversity': True,
    'ensure_test_variant_diversity': False,

    # === Sampling Configuration ===
    'sequential_sampling': True,   # True: sample from activity runs, False: uniform sampling
    'n_indices': None,             # Required when sequential_sampling=False
    'n_per_run': 7,                # Required when sequential_sampling=True
    'independent_sampling': True,

    # === Conformance Parameters ===
    'prob_threshold': 1e-6,                    # Probability threshold for filtering activities
    'chunk_size': 11,                          # Window size for chunked conformance
    'conformance_switch_penalty_weight': 1.0,  # Multiplier for label-switch penalty (actual penalty = weight * P(stay_with_prev_activity))
    'max_hist_len': 3,                         # History length for probability dict
    'merge_mismatched_boundaries': False,      # Merge adjacent chunks when boundary labels disagree
    

    # === Cost Function ===
    'cost_function': "linear",    # "linear", "logarithmic", or callable
    'model_move_cost': 1.0,
    'log_move_cost': 1.0,
    'tau_move_cost': 0.0,
    'non_sync_penalty': 1.0,

    # === Temperature Calibration ===
    'use_calibration': True,
    'temp_bounds': (1.0, 10.0),
    'temperature': None,

    # === Logging ===
    'verbose': True,
    'log_level': logging.INFO,

    # === Miscellaneous ===
    'round_precision': 2,
    'random_seed': 101,
    'save_model_path': "./discovered_petri_net",
    'save_model': True,
}

# Usage:
output = incremental_softmax_recovery(
    df=df,
    softmax_lst=softmax_lst,
    **config 
)

# Unpack results
results_df, accuracy_dict, prob_dict = output

2025-09-09 11:38:18,122 - incremental_softmax_recovery - INFO - Starting incremental softmax recovery (conformance-only).
2025-09-09 11:38:18,229 - incremental_softmax_recovery - INFO - Validated sequential case IDs (found 40 unique cases) and 40 softmax matrices.
2025-09-09 11:38:18,231 - incremental_softmax_recovery - INFO - Validated sampling parameters: sequential runs with n_per_run=7.
2025-09-09 11:38:18,232 - incremental_softmax_recovery - INFO - Validated input parameters: round_precision=2, prob_threshold=1e-06.
2025-09-09 11:38:18,235 - incremental_softmax_recovery - INFO - Prepared cost function: linear (model=1.0, log=1.0, tau=0.0).
2025-09-09 11:38:18,236 - incremental_softmax_recovery - INFO - Prepared softmax arrays: 40 traces with individual shape (19, 5687).
2025-09-09 11:38:19,189 - incremental_softmax_recovery - INFO - Filtered log and softmax matrices: 237820 -> 5852 events (2.5% retained).
2025-09-09 11:38:19,317 - incremental_softmax_recovery - INFO - Performed tr

PNG visualization saved to: discovered_petri_net.png
PDF visualization saved to: discovered_petri_net.pdf


2025-09-09 11:39:50,963 - classes - INFO - Built marking transition map with 1450 markings
2025-09-09 11:39:50,963 - incremental_softmax_recovery - INFO - Computed marking-to-transition map with 1450 reachable markings.
2025-09-09 11:39:50,968 - incremental_softmax_recovery - INFO - Built conditional probability dictionary: 274 histories, avg 1.8 activities per history.
2025-09-09 11:39:51,389 - incremental_softmax_recovery - INFO - Prepared 10 test softmax matrices with calibration (temperature=1.54).
2025-09-09 11:39:51,390 - incremental_softmax_recovery - INFO - Extracted 10 test case IDs for processing.
2025-09-09 11:39:51,391 - incremental_softmax_recovery - DEBUG - Using adaptive chunk size: 7 (base: 11, complexity: 145)


case 1/10 — conformance

2025-09-09 11:39:51,392 - incremental_softmax_recovery - DEBUG - Processing test case 1/10 (20) using 'conformance'


...

2025-09-09 11:42:49,157 - classes - INFO - Conformance total 126 steps in 0.500s (251.9 steps/s) across 18 chunks
2025-09-09 11:42:49,169 - incremental_softmax_recovery - DEBUG - Case 9/10 (3) [conformance]: SKTR=0.968, Argmax=0.952, Sequence length=126


case 10/10 — conformance

2025-09-09 11:42:49,169 - incremental_softmax_recovery - DEBUG - Processing test case 10/10 (18) using 'conformance'


2025-09-09 11:43:03,214 - classes - INFO - Conformance total 133 steps in 14.043s (9.5 steps/s) across 19 chunks
2025-09-09 11:43:03,246 - incremental_softmax_recovery - DEBUG - Case 10/10 (18) [conformance]: SKTR=0.767, Argmax=0.699, Sequence length=133


2025-09-09 11:43:03,250 - incremental_softmax_recovery - INFO - Built results DataFrame and accuracy dictionary.
2025-09-09 11:43:03,251 - incremental_softmax_recovery - INFO - Softmax trace recovery completed using conformance method.


In [9]:
# standard window-basedconformance 
accuracies = compute_accuracies_by_case(results_df)

accuracies

,case:concept:name,sktr_accuracy,argmax_accuracy
0,11,0.807453,0.813665
1,14,0.884354,0.884354
2,15,0.870748,0.863946
3,18,0.766917,0.699248
4,20,0.941558,0.902597
5,3,0.968254,0.952381
6,30,0.896825,0.904762
7,36,0.676190,0.666667
8,4,0.662857,0.657143
9,5,0.685714,0.668571


In [16]:
prob_dict[('1',)]

{'1': 0.86,
 '0': 0.03,
 '2': 0.03,
 '11': 0.02,
 '15': 0.03,
 '4': 0.03,
 '13': 0.01}

In [10]:
# Show all rows for case '28' without truncation, including full list values
case_28_df = results_df[results_df['case:concept:name'] == '36']
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None, 'display.width', None):
    display(case_28_df)
total_cost = case_28_df['sktr_move_cost'].sum()
print(f"Total SKTR move cost for case 28: {total_cost:.4f}")

,case:concept:name,step,sktr_activity,argmax_activity,ground_truth,all_probs,all_activities,is_correct,cumulative_accuracy,sktr_move_cost
490,36,0,17,17,17,"[0.0, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.03, 0.0, 0.01, 0.0, 0.01, 0.0, 0.88, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.12
491,36,1,17,17,17,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01, 0.0, 0.01, 0.01, 0.0, 0.0, 0.0, 0.01, 0.0, 0.94, 0.0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.06
492,36,2,17,17,17,"[0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.01, 0.0, 0.01, 0.0, 0.01, 0.0, 0.91, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.09
493,36,3,17,17,17,"[0.05, 0.01, 0.03, 0.02, 0.02, 0.02, 0.03, 0.02, 0.09, 0.05, 0.02, 0.07, 0.02, 0.03, 0.01, 0.01, 0.01, 0.47, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.53
494,36,4,17,17,17,"[0.02, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01, 0.02, 0.01, 0.01, 0.03, 0.0, 0.02, 0.0, 0.01, 0.0, 0.8, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.20
495,36,5,17,17,17,"[0.02, 0.01, 0.01, 0.01, 0.0, 0.01, 0.01, 0.01, 0.02, 0.01, 0.01, 0.04, 0.01, 0.01, 0.0, 0.01, 0.0, 0.82, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.18
496,36,6,17,17,17,"[0.02, 0.01, 0.02, 0.01, 0.01, 0.02, 0.02, 0.01, 0.07, 0.01, 0.02, 0.05, 0.01, 0.02, 0.0, 0.01, 0.0, 0.65, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,1.000000,0.35
497,36,0,17,17,0,"[0.19, 0.01, 0.03, 0.02, 0.01, 0.02, 0.02, 0.02, 0.06, 0.02, 0.04, 0.13, 0.04, 0.02, 0.0, 0.02, 0.01, 0.29, 0.02]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",False,0.875000,0.71
498,36,1,0,0,0,"[0.65, 0.03, 0.03, 0.01, 0.02, 0.01, 0.01, 0.01, 0.02, 0.02, 0.03, 0.03, 0.05, 0.01, 0.01, 0.01, 0.02, 0.02, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,0.888889,1.21
499,36,2,0,0,0,"[0.69, 0.03, 0.02, 0.01, 0.02, 0.01, 0.0, 0.01, 0.02, 0.01, 0.06, 0.02, 0.05, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]",True,0.900000,0.31


Total SKTR move cost for case 28: 56.9400
